In [2]:
# %cd dataset_220525
# %pwd
# !aws s3 cp s3://poc-2208-sorin/220525_불량분류_1차/ . --recursive
# !pip install opencv-python-headless

In [3]:
import os
base_dir = '/root/MZC_Yubin/Sorin'
train_dir = os.path.join(base_dir, 'dataset_220524')
validation_dir = os.path.join(base_dir, 'dataset_220525')

# 훈련에 사용되는 고양이/개 이미지 경로
train_etc = os.path.join(train_dir, 'B_ETC')
train_hole = os.path.join(train_dir, 'B_Hole')
train_swell_nomal = os.path.join(train_dir, 'B_Swell_Nomal')
train_swell_point = os.path.join(train_dir, 'B_Swell_pointed')
train_soot = os.path.join(train_dir, 'B_Swoot')
train_bad = os.path.join(train_dir, 'Bad')

# 테스트에 사용되는 고양이/개 이미지 경로
valid_etc = os.path.join(validation_dir, 'B_ETC')
valid_hole = os.path.join(validation_dir, 'B_Hole')
valid_swell_nomal = os.path.join(validation_dir, 'B_Swell_Nomal')
valid_swell_point = os.path.join(validation_dir, 'B_Swell_pointed')
valid_soot = os.path.join(validation_dir, 'B_Swoot')
valid_bad = os.path.join(validation_dir, 'Bad')

In [4]:
label_list = ['etc', 'hole', 'swell_nomal', 'swell_point', 'soot', 'bad']
# label_list = ['hole', 'soot', 'swell_nomal', 'swell_point']

for i in label_list:
    print(f'Total training {i} images :', len(os.listdir(globals()['train_{}'.format(i)])))
    print(f'Total validation {i} images :', len(os.listdir(globals()['valid_{}'.format(i)])))
    print("*"*10)

Total training etc images : 8
Total validation etc images : 17
**********
Total training hole images : 14
Total validation hole images : 48
**********
Total training swell_nomal images : 1
Total validation swell_nomal images : 15
**********
Total training swell_point images : 7
Total validation swell_point images : 27
**********
Total training soot images : 72
Total validation soot images : 12
**********
Total training bad images : 42
Total validation bad images : 13
**********


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rotation_range = 20, width_shift_range = 0.2, height_shift_range = 0.2, rescale = 1. / 255)
test_datagen  = ImageDataGenerator(rotation_range = 20, width_shift_range = 0.2, height_shift_range = 0.2, rescale = 1. / 255)

train_generator = train_datagen.flow_from_directory(train_dir, class_mode='sparse', target_size=(512,512))
validation_generator =  test_datagen.flow_from_directory(validation_dir, class_mode  = 'sparse', target_size =(512,512))

Found 145 images belonging to 6 classes.
Found 132 images belonging to 6 classes.


In [6]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
images = []

for i in enumerate(range(3)):
    img, label = train_generator.next()
    n_img = len(label)
    
    base = cv2.cvtColor(img[0], cv2.COLOR_RGB2BGR)  # keras는 RGB, openCV는 BGR이라 변경함
    for idx in range(n_img - 1):
        img2 = cv2.cvtColor(img[idx + 1], cv2.COLOR_RGB2BGR)
        base = np.hstack((base, img2))
    images.append(base)

img = images[0]
# print(images)
# for idx in range(len(images) - 1):
#     img = np.vstack((img, images[idx + 1]))
# plt.imshow(img)
# plt.show()

In [7]:
images[0].shape

(512, 16384, 3)

In [8]:
import tensorflow as tf

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(512, 512, 3)),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(len(label_list), activation='sigmoid')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
# model.summary()

In [9]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [10]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5,restore_best_weights=True, mode='min')

history = model.fit(train_generator,
                    validation_data=validation_generator,
                    epochs=100,
                    batch_size = 128,
                    verbose=1,
                    callbacks = early_stopping_callback)

Epoch 1/100
Extension horovod.torch has not been built: /usr/local/lib/python3.8/site-packages/horovod/torch/mpi_lib/_mpi_lib.cpython-38-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still avaiable.
[2022-09-15 04:21:19.619 tensorflow-2-6-gpu--ml-g4dn-xlarge-7eba0e151157af0bd8a25c896f03:20 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-09-15 04:21:19.646 tensorflow-2-6-gpu--ml-g4dn-xlarge-7eba0e151157af0bd8a25c896f03:20 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
5/5 [==============================] - 37s 7s/step - loss: 5.7421 - accuracy: 0.5241 - val_loss: 10.8270 - val_accuracy: 0.1591
Epoch 2/100
5/5 [==============================] - 21s 5s/step - loss: 1.7415 - accuracy: 0.5862 - val_loss: 2.7065 - val_accuracy: 0.1288
Epoch 3/100
1/5 [=====>........

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'go', label='Training Loss')
plt.plot(epochs, val_loss, 'g', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()